In [1]:
!pip install PyPDF2

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import PyPDF2 as pypdf
import regex as re
import numpy as np
import csv
import pandas as pd

In [2]:
PDFfile = open("2020-2022Tohono.pdf", "rb")
pdfread = pypdf.PdfFileReader(PDFfile)
pdfread

In [10]:
y = pdfread.getPage(0)
y.extractText().replace('\n', '')

"Tohono O'odham Community College - 2020-2022 (v. 20.0) 115 Accounting (ACC) ACC 100: Practical Accounting Procedures   Prerequisites: None      Credits: 3 cr. hrs. (3 pds: 3 lec)    Course Description: Introduction to accounting systems for small businesses. Includes the basic accounting cycle, use of special journals, procedures for controlling cash, and payroll accounting.   ACC 101: Financial Accounting    Prerequisites: None   Credits: 3 cr. hrs. (3 pds: 3 lec)  Course Description: Introduces accounting as a service activity, analytical discipline, and information system. Includes quantitative information to make decisions, identification of events that characterize economic activity, and the collection and communication of economic activity. Also includes recording   ACC 102: Managerial Accounting  Prerequisites: ACC 101 & MAT 089 or higher     Credits: 3 cr. hrs. (3 pds: 3 lec)    Course Description: Accounting training for managers. Includes concepts for those who are inside an

In [19]:
test = ''
for i in np.arange(0, pdfread.getNumPages()):
    test += pdfread.getPage(i).extractText().replace('\n', '')
test

'Tohono O\'odham Community College - 2020-2022 (v. 20.0) 115 Accounting (ACC) ACC 100: Practical Accounting Procedures   Prerequisites: None      Credits: 3 cr. hrs. (3 pds: 3 lec)    Course Description: Introduction to accounting systems for small businesses. Includes the basic accounting cycle, use of special journals, procedures for controlling cash, and payroll accounting.   ACC 101: Financial Accounting    Prerequisites: None   Credits: 3 cr. hrs. (3 pds: 3 lec)  Course Description: Introduces accounting as a service activity, analytical discipline, and information system. Includes quantitative information to make decisions, identification of events that characterize economic activity, and the collection and communication of economic activity. Also includes recording   ACC 102: Managerial Accounting  Prerequisites: ACC 101 & MAT 089 or higher     Credits: 3 cr. hrs. (3 pds: 3 lec)    Course Description: Accounting training for managers. Includes concepts for those who are inside a

In [5]:
pdfread.getNumPages()

64

In [17]:
def get_data(csv_name, pdf_name, pattern, dep_group=1, abbrev_group=2, c_group=3, 
             t_group=4, desc_group=5, year=None, grad_status=None, newline_repl=''):
    """Takes in name of PDF in local folder and extracts data using regex expression 
    PATTERN and given group numbers. YEAR is the catalog year.
    GRAD_STATUS is either None (if unknown), 'G', 'UG', or a number indicating at
    which number is the course graduate."""
    
    PDFfile = open(pdf_name, "rb")
    pdfread = pypdf.PdfFileReader(PDFfile)
    
    csv_writer = csv.writer(open(csv_name, 'w'))
    csv_writer.writerow(["Department Name", "Course Catalogue Number", "Course Name",
     "Course Description", "Graduate/Undergraduate", "Format", "Lab", 
     "Academic Catalogue Year"])
    FS_keys = ["agri", "food", "animal"]
    
    look_ahead = pattern + r'(.+?)(?=' + pattern + r')'
    
    dep_names = {}
    #test = page_range[0] #DEBUGGING
    text = ''
    for i in np.arange(0, pdfread.getNumPages()):
    #    print(test) #DEBUGGING
    #    test += 1 #DEBUGGING
        text += pdfread.getPage(i).extractText().replace('\n', newline_repl)
    matches = re.findall(look_ahead, text)
    last = re.sub(look_ahead, '', text)
    last = re.search(pattern + r'(.+)', last)
    matches = matches + [last.groups()]

    for match in matches:  
        #print(match) #DEBUGGING
        # Getting full department name
        if not dep_names.get(match[abbrev_group - 1]):
            dep_names[match[abbrev_group - 1]] = match[dep_group - 1]
        dep_name = dep_names.get(match[abbrev_group - 1])

        # Other groups
        course_num = int(re.sub(r'[a-zA-Z]+', '', match[c_group - 1]))
        title = str(match[t_group - 1])
        desc = str(match[desc_group - 1]) # assuming description is group 5

        #Filtering
        #for key in FS_keys:
        #    if (re.search(key, title, flags=re.IGNORECASE) or 
        #        re.search(key, desc, flags=re.IGNORECASE)):
        G_UG = grad_status
        if grad_status:
            if type(grad_status) == int:
                G_UG = "UG"
                if int(t.group(1)) >= grad_status:
                    G_UG = "G"
        lab = False
        if (re.search(r'Laboratory|Lab', title, flags=re.IGNORECASE) or 
        re.search(r'Laboratory|Lab', desc, flags=re.IGNORECASE)):
            lab = True
        csv_writer.writerow([dep_name, course_num, title, desc, 
                                 G_UG, None, lab, year])
                                # Leaving format blank for now
                        
    #print(dep_names) #DEBUGGING
                
pattern = r'((([a-zA-Z]+\s)+)\(\w{2,3}\))?\s+([\/a-zA-Z]+)\s([\d]+\w?):\s([^\d]+)Prerequisite.+?Description:\s'
#Partial Test set:
get_data('2020-2022Tohono.csv', "2020-2022Tohono.pdf", pattern, 
         2, 4, 5, 6, 7, '2020-2022', 'UG', '')

In [18]:
pd.read_csv('2020-2022Tohono.csv')

,Department Name,Course Catalogue Number,Course Name,Course Description,Graduate/Undergraduate,Format,Lab,Academic Catalogue Year
0,Accounting,100,Practical Accounting Procedures,Introduction to accounting systems for small b...,UG,NaN,False,2020-2022
1,Accounting,101,Financial Accounting,"Introduces accounting as a service activity, a...",UG,NaN,False,2020-2022
2,Accounting,102,Managerial Accounting,Accounting training for managers. Includes con...,UG,NaN,False,2020-2022
3,American Indian Studies,101,Introduction to American Indian Studies I,Examination of the diversity of American Indi...,UG,NaN,False,2020-2022
4,American Indian Studies,200,Culture and History of Indigenous Peoples,This course introduces students to the foundat...,UG,NaN,False,2020-2022
5,American Indian Studies,260,Culture and History of American Indian Women,This course examines and appraises the histori...,UG,NaN,False,2020-2022
6,Agriculture and Natural Resources,102,Animal Science,This class will cover fundamental principles o...,UG,NaN,False,2020-2022
7,Agriculture and Natural Resources,111,,This course combines classroom and fieldwork t...,UG,NaN,True,2020-2022
8,Agriculture and Natural Resources,128,Plant Ecology of the Sonoran Region,This course will cover the fundamentals of eco...,UG,NaN,False,2020-2022
9,Agriculture and Natural Resources,130,Plant Science,"Introduces the principles of plant growth, dev...",UG,NaN,True,2020-2022


In [13]:
sample = '156  Course Descriptions  AFRICAN-AMERICAN STUDIES  AAS 303/ART 303: African-American Art History. A course designed to reveal contributions and accomplishments of African-American artists and the influence of traditional early African art on art of the world. Credit: 3 semester hours.  AAS 314/HIS 314: African-Americans in the United States. The history of African-Americans in the United States, their role in the development of this country, and their struggle for rights, privileges, and protections of citizenship. Prerequisite(s): HIS 201 and HIS 202, or consent of instructor. Credit: 3 semester hours.  AAS 323/MUS 323: African-American Music. A study of the development of African-American music, including spirituals, blues, jazz, symphonic literature, and miscellaneous songs. Open to non-Music majors. Credit: 3 semester hours  AAS 325/MUS 325: Jazz History and Literature. A survey of jazz, including its origins, major performers, and composers. All idioms and styles are studied using selected readings and recordings. Open to non-Music majors. Credit: 3 semester hours.  AAS 365/HIS 365: African History to 1870. A survey tracing the earliest beginnings of man in Africa and the development of sub-and trans-Saharan civilizations. Prerequisites: HIS 101 and HIS 102. Credit: 3 semester hours.  AAS 366/HIS 366: African History from 1870. The creation and development of European colonial empires in Africa and the subsequent rise of independent African states. Prerequisites: HIS 101 and HIS 102. Credit: 3 semester hours.  AAS 409/ENG 409: Literature of African-Americans. A study of representative literary works written by African Americans from    Phillis Wheatley   to the present. Prerequisite: ENG 211. Credit: 3 semester hours.  AAS 411/HIS 411: African-Americans in the Twentieth Century. A thorough study of the contributions of African Americans to American culture and their struggle for full citizenship in the United States. Prerequisites: HIS 201 and HIS 202. Credit: 3 semester hours.  ACCOUNTING  ACC 201: Principles of Accounting I. An introduction to current accounting theory and practice, including accounting concepts, principles, financial statements, the accounting cycle, and selected assets such as cash, receivables, inventory and long-term assets. Prerequisites: MAT 115 or higher, BUA 101 and BUA 102 with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 202: Principles of Accounting II. A continuation of ACC 201. A study of accounting fundamentals including current liabilities, partnerships, corporations, the statement of cash flows, investments and financial performance evaluation, and accounting information for managerial decision-making. Prerequisites: ACC 201, BUA 101 and BUA 102, all with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 300: Managerial Accounting. An exploration of decision-making tools and use of accounting information used by internal management for directing Prerequisites: ACC 201, BUA 101 and BUA 102, all with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 301: Intermediate Accounting I. A comprehensive study of concepts and principles underlying corporate financial accounting and reporting. Emphasis is given to current assets and operating assets. Prerequisites: ACC 202, BUA 101 and BUA 102, all with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 302: Intermediate Accounting II. A study of investments, current and long-term liabilities, changes and error analysis, statement of cash flows, and interperiod income tax allocation. Prerequisite: ACC 301. A continuation of ACC 301. Credit: 3 semester hours.  ACC 303: Intermediate Accounting III. This course is a continuation of ACC 301 and 302. The course consists of a comprehensive study of concepts and principles underlying current financial accounting and reporting practices, including IFRS. Prerequisites: ACC 302 with grade of C or better. Topics include pensions, leases, accounting changes and error analysis, accounting for income taxes, statement of cash flows, governmental and non-profit accounting, business combinations and consolidated financial statements. Credit: 3 semester hours.  ACC 308: Cost Accounting I. The study of cost behavior from a managerial perspective. The principles, practices, and procedures involved in collecting, assembling, analyzing, and controlling manufacturing costs are examined. Prerequisites: ACC 202, BUA 101 and BUA 102, all with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 324: Accounting Information Systems. An exploration of the flow of accounting information and '
test = '([A-Z\s?]+)?\s{2}([A-Z]{3,4})\s+(\d{3,4})\/?(.+?):?(.+?)\.'
rest = re.sub(test + r'(.+?)(?=' + test + r')', '',sample)
rest

'156  Course Descriptions  AFRICAN-  ACC 324: Accounting Information Systems. An exploration of the flow of accounting information and '

In [18]:
rest_match = re.search(cmn_pattern + r'(.+)', rest)
print(rest_match.group(2))
print(rest_match.group(3))
print(rest_match.group(4))
print(rest_match.group(5))
print(rest_match.group(7))
rest_match.groups()

None
HCA
2533
Utilization of Community Resources
Students will be taught the value of fostering good community relations that directly impact program services. The function of various community services will be assessed as well as the study of those resources to enhance the quality of assistance to clients.


(None,
 None,
 'HCA',
 '2533',
 'Utilization of Community Resources',
 ' be',
 'Students will be taught the value of fostering good community relations that directly impact program services. The function of various community services will be assessed as well as the study of those resources to enhance the quality of assistance to clients.')

In [15]:
sample = '20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION   ACCOUNTING (ACC)   ACC 100 Fundamentals of Accounting (3)   Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a ccounting; fund accounts; and nonprofit accounting for  entities such as public colleges, universities and health care providers .   ACC 350 Finance (3)    ACC 310,  BUS 265, ECO 200 or 201.   Introduction to the theories and applications associated with the functions of capital markets. This  includes the conceptual foundations of portfolio theory, risk management, and asset valuation. This  course will examine the markets for equity, money, and  bonds. One hour laboratory required.   AGRICULTURE (AGR)   AGR 103 Conservation Planning (4)   This course introduces students to the understanding natural resource conservation issues and concerns  to allow them to develop conservation plans necessary to improve   natural resource conditions applicable  to tribal lands.   AGR 105 Int roduction to Animal Science (4)   This course will introduce students to animal and livestock industries, production, systems and markets.  Students will not only learn about national product ion but include Navajo Nation livestock industry.   AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '
dine_pattern = r'(\w+\s\(\w{2,3}\))?\s+([a-zA-Z]{2,3})\s([\d]+\w?)\s([^\d]+)\(\d\)\s+(.+?)'
last = re.sub(dine_pattern + '(?=' + dine_pattern + ')', '', sample)
l

'20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION      AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '

In [9]:
r'test' + r'test2'

'testtest2'

In [15]:
dine_pattern + r'(?=' + dine_pattern + r')'

'(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?)(?=(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?))'

In [33]:
sample
re.findall(dine_pattern + r'(.+)(?=' + dine_pattern + r')', sample)[0]

('ACCOUNTING (ACC)',
 'ACC',
 '100',
 'Fundamentals of Accounting ',
 'Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a

In [16]:
re.sub(r'[a-zA-Z]+', '', '102N')

'102'